# 效率分析

## 目标和前提条件

如何使用数学优化来衡量组织的效率？在本示例中，您将学习如何使用Gurobi Python API将效率分析模型表述为线性规划问题，然后使用Gurobi优化器生成最优解。

这个模型是H. Paul Williams所著《数学规划中的模型构建》第五版第278-280页和335-336页的示例22。

这个例子属于中级水平，我们假设您了解Python和Gurobi Python API，并且对构建数学优化模型有一定的了解。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 背景

数据包络分析(DEA)是运筹学和经济学中的一个非参数问题，其解是生产前沿的估计。它用于实证测量决策单元(DMUs)的生产效率。DEA问题有许多线性规划表述。Fuller的论文(1957)、Charnes等人(1978)和Thanassoulis等人(1987)对该主题进行了更全面的介绍。H.P. Williams给出的表述在Land(1991)中有所描述。这个表述是一个常用模型的对偶模型，该模型依赖于寻找输出与输入的加权比率。我们将使用Cooper等人(2007)中常用的表述。

数据包络分析已被用于评估许多不同国家中从事不同活动的各类实体的表现。例如，美国空军在不同地理位置的基地的维护活动，英格兰和威尔士的警察部队，以及塞浦路斯和加拿大的分行银行，以及美国、英国和法国的大学在执行教育和研究功能方面的效率。

DEA方法关注*效率*的评估。最常见的效率衡量采用以下比率形式：

$$
\text{效率} = \frac{\text{输出}}{\text{输入}}
$$


## 模型表述

假设有一组DMUs。为每个DMU选择一些常见的输入和输出项，如下所示：
1. 每个输入和输出都有可用的数值数据，假设所有DMU的数据都是正数。
2. 项目（输入、输出和DMU的选择）应反映分析师或经理对将进入DMU相对效率评估的组件的兴趣。
3. 原则上，较小的输入量是可取的，较大的输出量是可取的，因此效率分数应反映这些原则。
4. 不同输入和输出的计量单位不需要一致。有些可能涉及人数、楼面面积、花费的金钱等。

### 分数问题表述
目标DMU $k$ 的效率度量是通过加权输出与加权输入的比率的最大值来获得的，条件是每个DMU的类似比率小于或等于1。

### 集合和索引

$j,k \in \text{DMUS}$: DMU的索引和集合，其中$k$表示目标DMU。

$i \in \text{Inputs}$: 输入的索引和集合。

$r \in \text{Outputs}$: 输出的索引和集合。

### 参数

$\text{invalue}_{i,j} > 0$: DMU $j$ 的输入 $i$ 的值。

$\text{outvalue}_{r,j} > 0$: DMU $j$ 的输出 $r$ 的值。

### 决策变量

$u_{r} \geq 0$: 输出 $r$ 的权重。

$v_{i} \geq 0$: 输入 $i$ 的权重。

### 目标函数

**目标DMU效率**: 最大化目标DMU $k$ 的效率。

$$
\text{Maximize} \quad E_k = 
\frac{\sum_{r \in \text{Outputs}} \text{outvalue}_{r,k}*u_{r}}{\sum_{i \in \text{Inputs}} \text{invalue}_{i,k}*v_{i}}
\tag{FP0}
$$


### 约束条件

**效率比率**: DMU的效率是一个介于$[0,1]$之间的数值。

\begin{equation}
\frac{\sum_{r \in \text{Outputs}} \text{outvalue}_{r,j}*u_{r}}{\sum_{i \in \text{Inputs}} \text{invalue}_{i,j}*v_{i}}
 \leq 1 \quad \forall j \in \text{DMUS}
 \tag{FP1}
\end{equation}


### 线性规划问题表述

这个线性规划表述可以在Cooper等人(2007)的书中找到。

### 目标函数

**目标DMU效率**: 最大化目标DMU $k$ 的效率。

$$
\text{Maximize} \quad E_k = \sum_{r \in \text{Outputs}} \text{outvalue}_{r,k}*u_{r}
\tag{LP0}
$$


### 约束条件

**效率比率**: DMU的效率是一个介于$[0,1]$之间的数值。

\begin{equation}
\sum_{r \in \text{Outputs}} \text{outvalue}_{r,j}*u_{r} -
\sum_{i \in \text{Inputs}} \text{invalue}_{i,k}*v_{i}
 \leq 0  \quad \forall j \in \text{DMUS}
\tag{LP1}
\end{equation}

**归一化**: 这个约束确保分数问题目标函数的分母等于1。

\begin{equation}
\sum_{i \in \text{Inputs}} \text{invalue}_{i,k}*v_{i} = 1 
\tag{LP2}
\end{equation}

很容易验证分数问题和线性规划问题是等价的。假设分数问题的效率比率约束的分母对于所有DMU都是正数，那么我们可以通过将约束$FP1$的两边乘以分母来获得约束$LP1$。接下来，我们将$FP0$的分母设为1并定义约束$LP2$，然后最大化分子，得到目标函数$LP0$。

### 效率定义

1. 如果最优目标函数值 $E_{k}^{*} = 1$，则$DMU_k$是有效的。
2. 否则，$DMU_k$是无效的。

## 问题描述

一家汽车制造商希望评估获得其汽车销售许可证的不同车库的效率。每个车库都有一定数量的可测量“输入”：
* 员工
* 展厅面积
* 类别1的人口
* 类别2的人口
* Alpha型号的询问
* Beta型号的询问

每个车库也有一定数量的可测量“输出”：
* 不同品牌汽车的销售数量
* 年利润

下表给出了28个特许车库的输入和输出。

![inputOutput1](inputOutput1.PNG)
![inputOutput2](inputOutput2.PNG)

目标是识别有效和无效的车库及其输入输出权重。为了解决这个问题，有必要为每个车库求解LP模型。

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

### 辅助函数

* `solve_DEA` 构建并求解LP模型。

In [ ]:
%pip install gurobipy

In [ ]:
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# 测试环境：Python 3.7.0 & Gurobi 9.1.0

In [ ]:
def solve_DEA(target, verbose=True):
    # 车库的输入输出值
    inattr = ['staff', 'showRoom', 'Population1', 'Population2', 'alphaEnquiries', 'betaEnquiries']
    outattr = ['alphaSales', 'BetaSales', 'profit']
    
    dmus, inputs, outputs = gp.multidict({
        'Winchester': [{'staff': 7, 'showRoom': 8, 'Population1': 10, 'Population2': 12, 'alphaEnquiries': 8.5, 'betaEnquiries': 4}, {'alphaSales': 2, 'BetaSales': 0.6, 'profit': 1.5}],
        'Andover': [{'staff': 6, 'showRoom': 6, 'Population1': 20, 'Population2': 30, 'alphaEnquiries': 9, 'betaEnquiries': 4.5}, {'alphaSales': 2.3, 'BetaSales': 0.7, 'profit': 1.6}],
        'Basingstoke': [{'staff': 2, 'showRoom': 3, 'Population1': 40, 'Population2': 40, 'alphaEnquiries': 2, 'betaEnquiries': 1.5}, {'alphaSales': 0.8, 'BetaSales': 0.25, 'profit': 0.5}],
        'Poole': [{'staff': 14, 'showRoom': 9, 'Population1': 20, 'Population2': 25, 'alphaEnquiries': 10, 'betaEnquiries': 6}, {'alphaSales': 2.6, 'BetaSales': 0.86, 'profit': 1.9}],
        'Woking': [{'staff': 10, 'showRoom': 9, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 11, 'betaEnquiries': 5}, {'alphaSales': 2.4, 'BetaSales': 1, 'profit': 2}],
        'Newbury': [{'staff': 24, 'showRoom': 15, 'Population1': 15, 'Population2': 13, 'alphaEnquiries': 25, 'betaEnquiries': 1.9}, {'alphaSales': 8, 'BetaSales': 2.6, 'profit': 4.5}],
        'Portsmouth': [{'staff': 6, 'showRoom': 7, 'Population1': 50, 'Population2': 40, 'alphaEnquiries': 8.5, 'betaEnquiries': 3}, {'alphaSales': 2.5, 'BetaSales': 0.9, 'profit': 1.6}],
        'Alresford': [{'staff': 8, 'showRoom': 7.5, 'Population1': 5, 'Population2': 8, 'alphaEnquiries': 9, 'betaEnquiries': 4}, {'alphaSales': 2.1, 'BetaSales': 0.85, 'profit': 2}],
        'Salisbury': [{'staff': 5, 'showRoom': 5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 5, 'betaEnquiries': 2.5}, {'alphaSales': 2, 'BetaSales': 0.65, 'profit': 0.9}],
        'Guildford': [{'staff': 8, 'showRoom': 10, 'Population1': 30, 'Population2': 35, 'alphaEnquiries': 9.5, 'betaEnquiries': 4.5}, {'alphaSales': 2.05, 'BetaSales': 0.75, 'profit': 1.7}],
        'Alton': [{'staff': 7, 'showRoom': 8, 'Population1': 7, 'Population2': 8, 'alphaEnquiries': 3, 'betaEnquiries': 2}, {'alphaSales': 1.9, 'BetaSales': 0.70, 'profit': 0.5}],
        'Weybridge': [{'staff': 5, 'showRoom': 6.5, 'Population1': 9, 'Population2': 12, 'alphaEnquiries': 8, 'betaEnquiries': 4.5}, {'alphaSales': 1.8, 'BetaSales': 0.63, 'profit': 1.4}],
        'Dorchester': [{'staff': 6, 'showRoom': 7.5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 7.5, 'betaEnquiries': 4}, {'alphaSales': 1.5, 'BetaSales': 0.45, 'profit': 1.45}],
        'Bridport': [{'staff': 11, 'showRoom': 8, 'Population1': 8, 'Population2': 10, 'alphaEnquiries': 10, 'betaEnquiries': 6}, {'alphaSales': 2.2, 'BetaSales': 0.65, 'profit': 2.2}],
        'Weymouth': [{'staff': 4, 'showRoom': 5, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 7.5, 'betaEnquiries': 3.5}, {'alphaSales': 1.8, 'BetaSales': 0.62, 'profit': 1.6}],
        'Portland': [{'staff': 3, 'showRoom': 3.5, 'Population1': 3, 'Population2': 20, 'alphaEnquiries': 2, 'betaEnquiries': 1.5}, {'alphaSales': 0.9, 'BetaSales': 0.35, 'profit': 0.5}],
        'Chichester': [{'staff': 5, 'showRoom': 5.5, 'Population1': 8, 'Population2': 10, 'alphaEnquiries': 7, 'betaEnquiries': 3.5}, {'alphaSales': 1.2, 'BetaSales': 0.45, 'profit': 1.3}],
        'Petersfield': [{'staff': 21, 'showRoom': 12, 'Population1': 6, 'Population2': 6, 'alphaEnquiries': 15, 'betaEnquiries': 8}, {'alphaSales': 6, 'BetaSales': 0.25, 'profit': 2.9}],
        'Petworth': [{'staff': 6, 'showRoom': 5.5, 'Population1': 2, 'Population2': 2, 'alphaEnquiries': 8, 'betaEnquiries': 5}, {'alphaSales': 1.5, 'BetaSales': 0.55, 'profit': 1.55}],
        'Midhurst': [{'staff': 3, 'showRoom': 3.6, 'Population1': 3, 'Population2': 3, 'alphaEnquiries': 2.5, 'betaEnquiries': 1.5}, {'alphaSales': 0.8, 'BetaSales': 0.20, 'profit': 0.45}],
        'Reading': [{'staff': 30, 'showRoom': 29, 'Population1': 120, 'Population2': 80, 'alphaEnquiries': 35, 'betaEnquiries': 20}, {'alphaSales': 7, 'BetaSales': 2.5, 'profit': 8}],
        'Southampton': [{'staff': 25, 'showRoom': 16, 'Population1': 110, 'Population2': 80, 'alphaEnquiries': 27, 'betaEnquiries': 12}, {'alphaSales': 6.5, 'BetaSales': 3.5, 'profit': 5.4}],
        'Bournemouth': [{'staff': 19, 'showRoom': 10, 'Population1': 90, 'Population2': 22, 'alphaEnquiries': 25, 'betaEnquiries': 13}, {'alphaSales': 5.5, 'BetaSales': 3.1, 'profit': 4.5}],
        'Henley': [{'staff': 7, 'showRoom': 6, 'Population1': 5, 'Population2': 7, 'alphaEnquiries': 8.5, 'betaEnquiries': 4.5}, {'alphaSales': 1.2, 'BetaSales': 0.48, 'profit': 2}],
        'Maidenhead': [{'staff': 12, 'showRoom': 8, 'Population1': 7, 'Population2': 10, 'alphaEnquiries': 12, 'betaEnquiries': 7}, {'alphaSales': 4.5, 'BetaSales': 2, 'profit': 2.3}],
        'Fareham': [{'staff': 4, 'showRoom': 6, 'Population1': 1, 'Population2': 1, 'alphaEnquiries': 7.5, 'betaEnquiries': 3.5}, {'alphaSales': 1.1, 'BetaSales': 0.48, 'profit': 1.7}],
        'Romsey': [{'staff': 2, 'showRoom': 2.5, 'Population1': 1, 'Population2': 1, 'alphaEnquiries': 2.5, 'betaEnquiries': 1}, {'alphaSales': 0.4, 'BetaSales': 0.1, 'profit': 0.55}],
        'Ringwood': [{'staff': 2, 'showRoom': 3.5, 'Population1': 2, 'Population2': 2, 'alphaEnquiries': 1.9, 'betaEnquiries': 1.2}, {'alphaSales': 0.3, 'BetaSales': 0.09, 'profit': 0.4}]
    })
    
    ### 创建LP模型
    model = gp.Model('DEA')
    
    # 决策变量
    wout = model.addVars(outattr, name="outputWeight")
    win = model.addVars(inattr, name="inputWeight")

    # 约束条件
    ratios = model.addConstrs( ( gp.quicksum(outputs[h][r]*wout[r] for r in outattr ) 
                                - gp.quicksum(inputs[h][i]*win[i] for i in inattr ) 
                                <= 0 for h in dmus ), name='ratios' )
    
    normalization = model.addConstr((gp.quicksum(inputs[target][i]*win[i] for i in inattr ) == 1 ),
                                    name='normalization')
    
    # 目标函数
    
    model.setObjective( gp.quicksum(outputs[target][r]*wout[r] for r in outattr ), GRB.MAXIMIZE)
    
    # 运行优化引擎
    if not verbose:
        model.params.OutputFlag = 0
    model.optimize()
    
    # 打印结果
    print(f"\n目标DMU {target} 的效率是 {round(model.objVal,3)}") 
    
    print("__________________________________________________________________")
    print(f"输入的权重是：")
    for i in inattr:
        print(f"{i} 的权重是: {round(win[i].x,3)} ") 
        
    print("__________________________________________________________________")
    print(f"输出的权重是：")
    for r in outattr:
        print(f"{r} 的权重是: {round(wout[r].x,3)} ") 
    print("__________________________________________________________________\n\n")  
    
    return model.objVal

## 输入数据
我们定义车库列表。

In [3]:
dmus = ['Winchester','Andover','Basingstoke', 'Poole', 'Woking','Newbury','Portsmouth','Alresford','Salisbury','Guildford','Alton','Weybridge', 'Dorchester', 'Bridport', 'Weymouth', 'Portland', 'Chichester', 'Petersfield', 'Petworth', 'Midhurst', 'Reading', 'Southampton', 'Bournemouth', 'Henley', 'Maidenhead', 'Fareham', 'Romsey', 'Ringwood']

---
## 输出报告

我们打印出每个车库的效率分数及其相关的输入和输出权重。

In [ ]:
# 为每个DMU求解DEA模型

performance = {}
for h in dmus:    
    performance[h] = solve_DEA(h, verbose=False)


Using license file c:\gurobi\gurobi.lic

The efficiency of target DMU Winchester is 0.835
__________________________________________________________________
The weights for the inputs are:
For staff: 0.012 
For showRoom: 0.0 
For Population1: 0.0 
For Population2: 0.002 
For alphaEnquiries: 0.095 
For betaEnquiries: 0.02 
__________________________________________________________________
The weights for the outputs are
For alphaSales is: 0.113 
For BetaSales is: 0.004 
For profit is: 0.404 
__________________________________________________________________



The efficiency of target DMU Andover is 0.917
__________________________________________________________________
The weights for the inputs are:
For staff: 0.115 
For showRoom: 0.03 
For Population1: 0.0 
For Population2: 0.0 
For alphaEnquiries: 0.014 
For betaEnquiries: 0.0 
__________________________________________________________________
The weights for the outputs are
For alphaSales is: 0.399 
For BetaSales is: 0.0 
For prof

---
## 分析

我们识别哪些车库是有效的，哪些是无效的，并提供每个车库的效率分数。

In [ ]:
# 识别有效和无效的DMU

# 按效率分数降序排列车库
sorted_performance = {k: v for k, v in sorted(performance.items(), key=lambda item: item[1], reverse = True)}

efficient = []
inefficient = []

for h in sorted_performance.keys():
    if sorted_performance[h] >= 0.9999999:
        efficient.append(h) 
    if sorted_performance[h] < 0.9999999:
        inefficient.append(h) 
        
print('____________________________________________')
print(f"有效的DMU是：")
for eff in efficient:
    print(f"DMU {eff} 的效率分数是: {round(performance[eff],3)}") 
    
print('____________________________________________')
print(f"无效的DMU是：")
for ine in inefficient:
    print(f"DMU {ine} 的效率分数是: {round(performance[ine],3)}") 


____________________________________________
The efficient DMUs are:
The performance value of DMU Newbury is: 1.0
The performance value of DMU Alresford is: 1.0
The performance value of DMU Salisbury is: 1.0
The performance value of DMU Alton is: 1.0
The performance value of DMU Weymouth is: 1.0
The performance value of DMU Petersfield is: 1.0
The performance value of DMU Southampton is: 1.0
The performance value of DMU Bournemouth is: 1.0
The performance value of DMU Maidenhead is: 1.0
The performance value of DMU Fareham is: 1.0
The performance value of DMU Romsey is: 1.0
The performance value of DMU Basingstoke is: 1.0
The performance value of DMU Portsmouth is: 1.0
The performance value of DMU Portland is: 1.0
The performance value of DMU Henley is: 1.0
____________________________________________
The inefficient DMUs are:
The performance value of DMU Petworth is: 0.988
The performance value of DMU Reading is: 0.984
The performance value of DMU Bridport is: 0.982
The performance va

## 参考文献

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Cooper, W. W, L. M. Seiford, K. Tone. (2007) Data Envelopment Analysis: A Comprehensive Text with Models, Applications, References and DEA-Solver Software. Second edition. Springer-Verlag US.

Land, A. (1991) Data envelopment analysis, Chapter 5, in Operations Research in Management (eds S.C. Littlechild and M.F. Shutler), Prentice Hall, London.

Farrell, M.J. (1957) The measurement of productive efficiency. Journal of the Royal Statistical Society, Series A, 120, 253–290.

Charnes, A., Cooper, W.W. and Rhodes, E. (1978) Measuring the efficiency of decision making units. European Journal of Operational Research, 2, 429–444.

Thanassoulis, E., Dyson, R.G. and Foster, M.J. (1987) Relative efficiency assessments using data envelopment analysis: an application to data on rates departments. Journal of the Operational Research Society, 5, 397–411.

Copyright © 2020 Gurobi Optimization, LLC